In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.datasets import mnist

Using TensorFlow backend.


##### Loading our datasets :containing images of handwritten digits 0f 0 - 9

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#### Flatten the images to pas into the neural network 

In [4]:
img_width, img_height = 28, 28

In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_width, img_height)
    x_test = x_test.reshape(x_test.shape[0], 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else:
    x_train = x_train.reshape(x_train.shape[0], img_width, img_height, 1)
    x_test = x_test.reshape(x_test.shape[0], img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#### Normalizing our datasets

In [7]:
sx_train = x_train / 255
x_test = x_test / 255

train_y = to_categorical(y_train)
test_y = to_categorical(y_test)

print(x_train.shape)
print(x_test.shape)
print(train_y.shape)
print(test_y.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 10)
(10000, 10)


#### Build the model 

In [8]:
model = Sequential()
model.add(Convolution2D(filters = 32, kernel_size = (5, 5), input_shape = input_shape, activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Convolution2D(filters = 64, kernel_size = (5, 5), input_shape = input_shape, activation = "relu"))
model.add(MaxPooling2D(pool_size = (3, 3)))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))

##### Compiling our model 

In [10]:
model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",#classes that are greater than 2
              metrics = ["accuracy"])

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [11]:
model.fit(x_train, train_y, validation_data = (x_test, test_y),
          epochs = 12, #the no. of iteration over the entire dataset train on
          batch_size = 64)
scores = model.evaluate(x_test, test_y)
print("CNN Error : %.2f%%" % (100- scores[1] * 100))
model.save("model.mnist")

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 75s 1ms/step - loss: 0.6056 - accuracy: 0.8957 - val_loss: 2.2707 - val_accuracy: 0.4715
Epoch 2/12
60000/60000 [==============================] - 75s 1ms/step - loss: 0.1109 - accuracy: 0.9676 - val_loss: 2.2680 - val_accuracy: 0.6076
Epoch 3/12
60000/60000 [==============================] - 76s 1ms/step - loss: 0.0826 - accuracy: 0.9753 - val_loss: 2.2719 - val_accuracy: 0.3914
Epoch 4/12
60000/60000 [==============================] - 77s 1ms/step - loss: 0.0692 - accuracy: 0.9796 - val_loss: 2.2755 - val_accuracy: 0.3929
Epoch 5/12
60000/60000 [==============================] - 77s 1ms/step - loss: 0.0623 - accuracy: 0.9814 - val_loss: 2.2772 - val_accuracy: 0.3624
Epoch 6/12
60000/60000 [==============================] - 78s 1ms/step - loss: 0.0528 - accuracy: 0.9847 - val_loss: 2.2745 - val_accuracy: 0.2766
Epoch 7/12
60000/60000 [==============================] - 78s 1ms/st

In [13]:
#predict on the first 5 test images
predictions = model.predict(x_test[:5])
# print our model prediction
print(np.argmax(predictions,axis=1))
print(test_y[:5])

[1 1 1 1 1]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
